In [1]:
## %matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import glob
import sys
import tensorflow as tf
import math
import datetime

In [2]:
# Helper functions, DO NOT modify this

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)
        
# Functions to load data, DO NOT change these

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((y.shape[0], num_classes))
    y_one_hot[y] = 1
    return y_one_hot.T

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f).astype(np.float)
#         img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    # X = np.column_stack(images)
    X = np.stack(images, axis=0)
    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)

In [3]:
# Load the data
data_root_path = 'cifar10-hw/'
X_train, y_train = get_train_data(data_root_path) # this may take a few minutes
X_test = get_images(data_root_path + 'test')
print('Data loading done')

{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
Loaded 10000/10000
Data loading done


In [4]:
print(X_train.shape)
print(y_train.shape)

(50000, 32, 32, 3)
(50000,)


In [5]:
# Group's helper function
def split(X, y, val_size):
    '''
    split the data into training and validation set
    '''
    indices = np.random.permutation(X.shape[0])
    test_num = int(val_size * X.shape[0])
    return X[indices[test_num:]], X[indices[:test_num]], y[indices[test_num:]], y[indices[:test_num]]

In [6]:
X_trn, X_val, y_trn, y_val = split(X_train, y_train, val_size=0.1)
print(X_trn.shape)
print(X_val.shape)
print(y_trn.shape)
print(y_val.shape)

(45000, 32, 32, 3)
(5000, 32, 32, 3)
(45000,)
(5000,)


In [7]:
# GLobal value
H, W, T = 32, 32, 10 # height/width of images, number of classes of images
cnns = dict()

In [8]:
class CNN():
    def __init__(self, model_fn, trainer, global_step=None):
        if global_step is None:
            tf.reset_default_graph()
            global_step = tf.Variable(0, trainable=False)
        self.X = tf.placeholder(tf.float32, [None, H, W, 3])
        self.Y = tf.placeholder(tf.int64, [None])
        self.is_training = tf.placeholder(tf.bool)
        logit, loss = model_fn(self.X, self.Y, self.is_training)
        # https://stackoverflow.com/a/43285333
        extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(extra_update_ops):
            train_op = trainer.minimize(loss, global_step=global_step)
        # train_op = trainer.minimize(loss)
        # Accuracy
        correct = tf.equal(tf.argmax(logit, 1), self.Y)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
        self.variables = {
            'train': [loss, correct, train_op],
            'validate': [loss, correct, accuracy]}
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        
    def batch_gen(self, Xd, Yd, batch_size, shuffle=True):
        indicies = np.arange(Xd.shape[0])
        if shuffle:
            np.random.shuffle(indicies)
        for i in range(int(math.ceil(Xd.shape[0] / batch_size))):
            start_idx = (i * batch_size) % Xd.shape[0]
            idx = indicies[start_idx:start_idx + batch_size]
            yield Xd[idx, :], Yd[idx]

    def run(self, Xd, Yd, epochs, batch_size, print_every, plot_losses, status):
        iter_cnt = 0
        for e in range(epochs):
            correct = 0
            losses = []
            for Xb, Yb in self.batch_gen(Xd, Yd, batch_size, shuffle = True):
                feed_dict = {self.X: Xb, self.Y: Yb, self.is_training: status=='train'}
                loss, corr, _ = self.sess.run(self.variables[status], feed_dict = feed_dict)
                losses.append(loss)
                correct += np.sum(corr)
                if status == 'train' and iter_cnt % print_every == 0:
                    print("{} Iter {}: batch trn loss = {:.3f}, accuracy = {:.3f}".format(
                        datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                        iter_cnt,
                        loss,
                        np.mean(corr),
                    ))
                iter_cnt += 1
            epoch_loss = np.mean(losses)
            epoch_accuracy = correct / Xd.shape[0]
            print("Epoch {}: mean loss = {:.3f}, accuracy = {:.3f}".format(
                e, epoch_loss, epoch_accuracy))
            if plot_losses:
                plt.plot(losses)
                plt.grid(True)
                plt.title('Epoch {} Mean Loss'.format(e+1))
                plt.xlabel('minibatch number')
                plt.ylabel('minibatch mean loss')
                plt.show()
        return epoch_loss, epoch_accuracy

    def train(self, Xd, Yd, epochs=1, batch_size=50, print_every=100, plot_losses=False):
        return self.run(Xd, Yd, epochs, batch_size, print_every, plot_losses, status='train')
            
    def validate(self, Xd, Yd, epochs=1, batch_size=50, print_every=100, plot_losses=False):
        return self.run(Xd, Yd, epochs, batch_size, print_every, plot_losses, status='validate')
    
    def predict(self, Xd):
        feed_dict = {self.X: Xd, self.is_training: False}
        variables = [self.Y]
        Yp = self.sess.run(variables, feed_dict = feed_dict)
        return Yp

## [0102] witness
- base (0101): add winess, batch_size to 100, with dropout 0.5 (dropout is not working)

In [17]:
def witness(raw_images):
    images = np.empty_like(raw_images)
    np.copyto(images, raw_images)
    for i in range(images.shape[0]):
        old = images[i]
        new = (old - np.mean(old)) / np.std(old)
        images[i, :, :, :] = new
    return images

In [19]:
def model_fn(layer_input, labels, is_training):
    reg_scale= 0.1
    regularizer = tf.contrib.layers.l2_regularizer(scale=reg_scale)
    # initializer = tf.contrib.layers.xavier_initializer()
    initializer = None
    h, w = H, W
    # conv - bn - max_pool
    F1 = 64
    P1, S1 = 3, 2 # pool_size, strides
    layer = tf.layers.conv2d(
        inputs=layer_input, filters=F1, kernel_size=[5, 5], padding='same', activation=tf.nn.relu,
        kernel_regularizer=regularizer, kernel_initializer=initializer)
    layer = tf.layers.batch_normalization(
        inputs=layer, training=is_training,
        beta_regularizer=regularizer, gamma_regularizer=regularizer)
    layer = tf.layers.max_pooling2d(inputs=layer, pool_size=P1, strides=S1)
    h = math.floor((h - P1 + S1) / S1)
    w = math.floor((w - P1 + S1) / S1)
    # conv - bn - max_pool
    F2 = 64
    P2, S2 = 3, 2 # pool_size, strides
    layer = tf.layers.conv2d(
        inputs=layer, filters=F2, kernel_size=[5, 5], padding='same', activation=tf.nn.relu,
        kernel_regularizer=regularizer, kernel_initializer=initializer)
    layer = tf.layers.batch_normalization(
        inputs=layer, training=is_training,
        beta_regularizer=regularizer, gamma_regularizer=regularizer)
    layer = tf.layers.max_pooling2d(inputs=layer, pool_size=P2, strides=S2)
    h = math.floor((h - P2 + S2) / S2)
    w = math.floor((w - P2 + S2) / S2)
    # dense1 - bn - dropout - fc - softmax
    flat_size = F2 * h * w
    print(flat_size)
    layer = tf.reshape(layer, [-1, flat_size])
    layer = tf.layers.dense(
        inputs=layer, units=1024, activation=tf.nn.relu,
        kernel_regularizer=regularizer, kernel_initializer=initializer)
    layer = tf.layers.batch_normalization(
        inputs=layer, training=is_training,
        beta_regularizer=regularizer, gamma_regularizer=regularizer)
    layer = tf.layers.dropout(inputs=layer, rate=0.5)
    # no activation here for logit, as it will be calculated in loss
    logit = tf.layers.dense(
        inputs=layer, units=T, activation=None,
        kernel_regularizer=regularizer, kernel_initializer=initializer)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int64), depth=10)
    loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits=logit)
    return logit, loss

trainer = tf.train.GradientDescentOptimizer(learning_rate=0.03)
cnn = CNN(model_fn, trainer)
cnns['0102'] = cnn

m_X_trn = witness(X_trn)
m_X_val = witness(X_val)

3136


In [20]:
# 5 epochs
print('train')
cnn.train(m_X_trn, y_trn, epochs=5, batch_size=100, print_every=100, plot_losses=False)
print('validation')
cnn.validate(m_X_val, y_val, epochs=1, batch_size=y_val.shape[0])

train
2017-10-29 11:20:30 Iter 0: batch trn loss = 3.250, accuracy = 0.070
2017-10-29 11:21:40 Iter 100: batch trn loss = 1.315, accuracy = 0.530
2017-10-29 11:22:51 Iter 200: batch trn loss = 1.346, accuracy = 0.510
2017-10-29 11:23:57 Iter 300: batch trn loss = 1.078, accuracy = 0.640
2017-10-29 11:25:05 Iter 400: batch trn loss = 1.171, accuracy = 0.620
Epoch 0: mean loss = 1.329, accuracy = 0.550
2017-10-29 11:26:12 Iter 500: batch trn loss = 0.779, accuracy = 0.720
2017-10-29 11:27:19 Iter 600: batch trn loss = 1.067, accuracy = 0.580
2017-10-29 11:28:26 Iter 700: batch trn loss = 0.834, accuracy = 0.690
2017-10-29 11:29:33 Iter 800: batch trn loss = 0.763, accuracy = 0.760
Epoch 1: mean loss = 0.843, accuracy = 0.710
2017-10-29 11:30:40 Iter 900: batch trn loss = 0.543, accuracy = 0.790
2017-10-29 11:31:47 Iter 1000: batch trn loss = 0.804, accuracy = 0.720
2017-10-29 11:32:54 Iter 1100: batch trn loss = 0.628, accuracy = 0.760
2017-10-29 11:34:02 Iter 1200: batch trn loss = 0.62

(0.81093341, 0.72619999999999996)

In [21]:
# 10 epochs
print('train')
cnn.train(m_X_trn, y_trn, epochs=5, batch_size=100, print_every=100, plot_losses=False)
print('validation')
cnn.validate(m_X_val, y_val, epochs=1, batch_size=y_val.shape[0])

train
2017-10-29 11:45:56 Iter 0: batch trn loss = 0.269, accuracy = 0.930
2017-10-29 11:47:04 Iter 100: batch trn loss = 0.330, accuracy = 0.920
2017-10-29 11:48:12 Iter 200: batch trn loss = 0.439, accuracy = 0.850
2017-10-29 11:49:19 Iter 300: batch trn loss = 0.350, accuracy = 0.880
2017-10-29 11:50:24 Iter 400: batch trn loss = 0.419, accuracy = 0.880
Epoch 0: mean loss = 0.317, accuracy = 0.897
2017-10-29 11:51:29 Iter 500: batch trn loss = 0.212, accuracy = 0.960
2017-10-29 11:52:35 Iter 600: batch trn loss = 0.160, accuracy = 0.970
2017-10-29 11:53:43 Iter 700: batch trn loss = 0.156, accuracy = 0.980
2017-10-29 11:54:50 Iter 800: batch trn loss = 0.225, accuracy = 0.920
Epoch 1: mean loss = 0.234, accuracy = 0.927
2017-10-29 11:55:56 Iter 900: batch trn loss = 0.127, accuracy = 0.980
2017-10-29 11:57:02 Iter 1000: batch trn loss = 0.171, accuracy = 0.940
2017-10-29 11:58:09 Iter 1100: batch trn loss = 0.124, accuracy = 0.980
2017-10-29 11:59:16 Iter 1200: batch trn loss = 0.31

(0.82861483, 0.76400000000000001)

In [22]:
# 15 epochs
print('train')
cnn.train(m_X_trn, y_trn, epochs=5, batch_size=100, print_every=100, plot_losses=False)
print('validation')
cnn.validate(m_X_val, y_val, epochs=1, batch_size=y_val.shape[0])

train
2017-10-29 12:11:20 Iter 0: batch trn loss = 0.027, accuracy = 1.000
2017-10-29 12:12:26 Iter 100: batch trn loss = 0.040, accuracy = 1.000
2017-10-29 12:13:30 Iter 200: batch trn loss = 0.061, accuracy = 0.990
2017-10-29 12:14:34 Iter 300: batch trn loss = 0.063, accuracy = 1.000
2017-10-29 12:15:37 Iter 400: batch trn loss = 0.023, accuracy = 1.000
Epoch 0: mean loss = 0.041, accuracy = 0.996
2017-10-29 12:16:41 Iter 500: batch trn loss = 0.040, accuracy = 0.990
2017-10-29 12:17:45 Iter 600: batch trn loss = 0.027, accuracy = 1.000
2017-10-29 12:18:51 Iter 700: batch trn loss = 0.025, accuracy = 1.000
2017-10-29 12:19:58 Iter 800: batch trn loss = 0.023, accuracy = 1.000
Epoch 1: mean loss = 0.028, accuracy = 0.998
2017-10-29 12:21:04 Iter 900: batch trn loss = 0.024, accuracy = 1.000
2017-10-29 12:22:11 Iter 1000: batch trn loss = 0.033, accuracy = 1.000
2017-10-29 12:23:18 Iter 1100: batch trn loss = 0.025, accuracy = 1.000
2017-10-29 12:24:25 Iter 1200: batch trn loss = 0.01

(0.85706973, 0.77700000000000002)

## [0103] witness + dropout
- base (0101): add witness and dropout = 0.5
- batch size still 50
- trn: , val: (05 epochs)
- trn: , val: (10 epochs)
- trn: , val: (15 epochs)
- trn: , val: (20 epochs)

In [25]:
def model_fn(layer_input, labels, is_training):
    reg_scale= 0.1
    regularizer = tf.contrib.layers.l2_regularizer(scale=reg_scale)
    # initializer = tf.contrib.layers.xavier_initializer()
    initializer = None
    h, w = H, W
    # conv - bn - max_pool
    F1 = 64
    P1, S1 = 3, 2 # pool_size, strides
    layer = tf.layers.conv2d(
        inputs=layer_input, filters=F1, kernel_size=[5, 5], padding='same', activation=tf.nn.relu,
        kernel_regularizer=regularizer, kernel_initializer=initializer)
    layer = tf.layers.batch_normalization(
        inputs=layer, training=is_training,
        beta_regularizer=regularizer, gamma_regularizer=regularizer)
    layer = tf.layers.max_pooling2d(inputs=layer, pool_size=P1, strides=S1)
    h = math.floor((h - P1 + S1) / S1)
    w = math.floor((w - P1 + S1) / S1)
    # conv - bn - max_pool
    F2 = 64
    P2, S2 = 3, 2 # pool_size, strides
    layer = tf.layers.conv2d(
        inputs=layer, filters=F2, kernel_size=[5, 5], padding='same', activation=tf.nn.relu,
        kernel_regularizer=regularizer, kernel_initializer=initializer)
    layer = tf.layers.batch_normalization(
        inputs=layer, training=is_training,
        beta_regularizer=regularizer, gamma_regularizer=regularizer)
    layer = tf.layers.max_pooling2d(inputs=layer, pool_size=P2, strides=S2)
    h = math.floor((h - P2 + S2) / S2)
    w = math.floor((w - P2 + S2) / S2)
    # dense1 - bn - dropout - fc - softmax
    flat_size = F2 * h * w
    print(flat_size)
    layer = tf.reshape(layer, [-1, flat_size])
    layer = tf.layers.dense(
        inputs=layer, units=1024, activation=tf.nn.relu,
        kernel_regularizer=regularizer, kernel_initializer=initializer)
    layer = tf.layers.batch_normalization(
        inputs=layer, training=is_training,
        beta_regularizer=regularizer, gamma_regularizer=regularizer)
    layer = tf.layers.dropout(inputs=layer, rate=0.5, training=is_training)
    # no activation here for logit, as it will be calculated in loss
    logit = tf.layers.dense(
        inputs=layer, units=T, activation=None,
        kernel_regularizer=regularizer, kernel_initializer=initializer)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int64), depth=10)
    loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits=logit)
    return logit, loss

trainer = tf.train.GradientDescentOptimizer(learning_rate=0.03)
cnn = CNN(model_fn, trainer)
cnns['0103'] = cnn

m_X_trn = witness(X_trn)
m_X_val = witness(X_val)

3136


In [26]:
# 5 epochs
print('train')
cnn.train(m_X_trn, y_trn, epochs=5, batch_size=50, print_every=100, plot_losses=False)
print('validation')
cnn.validate(m_X_val, y_val, epochs=1, batch_size=y_val.shape[0])

train
2017-10-29 12:55:58 Iter 0: batch trn loss = 3.171, accuracy = 0.120
2017-10-29 12:56:31 Iter 100: batch trn loss = 1.984, accuracy = 0.360
2017-10-29 12:57:05 Iter 200: batch trn loss = 1.572, accuracy = 0.360
2017-10-29 12:57:40 Iter 300: batch trn loss = 1.628, accuracy = 0.520
2017-10-29 12:58:12 Iter 400: batch trn loss = 1.474, accuracy = 0.480
2017-10-29 12:58:44 Iter 500: batch trn loss = 1.485, accuracy = 0.480
2017-10-29 12:59:17 Iter 600: batch trn loss = 1.381, accuracy = 0.580
2017-10-29 12:59:49 Iter 700: batch trn loss = 1.681, accuracy = 0.440
2017-10-29 13:00:21 Iter 800: batch trn loss = 0.820, accuracy = 0.700
Epoch 0: mean loss = 1.607, accuracy = 0.486
2017-10-29 13:00:53 Iter 900: batch trn loss = 1.120, accuracy = 0.600
2017-10-29 13:01:26 Iter 1000: batch trn loss = 1.241, accuracy = 0.640
2017-10-29 13:01:58 Iter 1100: batch trn loss = 1.265, accuracy = 0.520
2017-10-29 13:02:31 Iter 1200: batch trn loss = 0.929, accuracy = 0.700
2017-10-29 13:03:03 Iter 

(0.77256513, 0.73980000000000001)

In [27]:
# 10 epochs
print('train')
cnn.train(m_X_trn, y_trn, epochs=5, batch_size=50, print_every=100, plot_losses=False)
print('validation')
cnn.validate(m_X_val, y_val, epochs=1, batch_size=y_val.shape[0])

train
2017-10-29 14:29:36 Iter 0: batch trn loss = 0.789, accuracy = 0.760
2017-10-29 14:30:11 Iter 100: batch trn loss = 0.295, accuracy = 0.900
2017-10-29 14:30:45 Iter 200: batch trn loss = 0.714, accuracy = 0.800
2017-10-29 14:31:18 Iter 300: batch trn loss = 0.900, accuracy = 0.680
2017-10-29 14:31:51 Iter 400: batch trn loss = 0.469, accuracy = 0.860
2017-10-29 14:32:26 Iter 500: batch trn loss = 0.980, accuracy = 0.720
2017-10-29 14:32:58 Iter 600: batch trn loss = 1.012, accuracy = 0.740
2017-10-29 14:33:31 Iter 700: batch trn loss = 0.525, accuracy = 0.820
2017-10-29 14:34:03 Iter 800: batch trn loss = 0.737, accuracy = 0.740
Epoch 0: mean loss = 0.683, accuracy = 0.768
2017-10-29 14:34:35 Iter 900: batch trn loss = 0.579, accuracy = 0.820
2017-10-29 14:35:08 Iter 1000: batch trn loss = 0.526, accuracy = 0.740
2017-10-29 14:35:40 Iter 1100: batch trn loss = 0.613, accuracy = 0.820
2017-10-29 14:36:12 Iter 1200: batch trn loss = 0.534, accuracy = 0.760
2017-10-29 14:36:44 Iter 

(0.69985479, 0.77359999999999995)

In [28]:
# 15 epochs
print('train')
cnn.train(m_X_trn, y_trn, epochs=5, batch_size=50, print_every=100, plot_losses=False)
print('validation')
cnn.validate(m_X_val, y_val, epochs=1, batch_size=y_val.shape[0])

train
2017-10-29 14:54:04 Iter 0: batch trn loss = 0.442, accuracy = 0.840
2017-10-29 14:54:36 Iter 100: batch trn loss = 0.553, accuracy = 0.860
2017-10-29 14:55:08 Iter 200: batch trn loss = 0.305, accuracy = 0.920
2017-10-29 14:55:40 Iter 300: batch trn loss = 0.427, accuracy = 0.880
2017-10-29 14:56:12 Iter 400: batch trn loss = 0.415, accuracy = 0.880
2017-10-29 14:56:44 Iter 500: batch trn loss = 0.399, accuracy = 0.820
2017-10-29 14:57:16 Iter 600: batch trn loss = 0.434, accuracy = 0.880
2017-10-29 14:57:48 Iter 700: batch trn loss = 0.358, accuracy = 0.860
2017-10-29 14:58:20 Iter 800: batch trn loss = 0.412, accuracy = 0.860
Epoch 0: mean loss = 0.441, accuracy = 0.845
2017-10-29 14:58:52 Iter 900: batch trn loss = 0.365, accuracy = 0.880
2017-10-29 14:59:24 Iter 1000: batch trn loss = 0.640, accuracy = 0.800
2017-10-29 14:59:56 Iter 1100: batch trn loss = 0.383, accuracy = 0.900
2017-10-29 15:00:28 Iter 1200: batch trn loss = 0.453, accuracy = 0.840
2017-10-29 15:01:00 Iter 

(0.70572072, 0.79359999999999997)

In [29]:
# 20 epochs
print('train')
cnn.train(m_X_trn, y_trn, epochs=5, batch_size=50, print_every=100, plot_losses=False)
print('validation')
cnn.validate(m_X_val, y_val, epochs=1, batch_size=y_val.shape[0])

train
2017-10-29 15:53:00 Iter 0: batch trn loss = 0.115, accuracy = 0.980
2017-10-29 15:53:34 Iter 100: batch trn loss = 0.162, accuracy = 0.980
2017-10-29 15:54:08 Iter 200: batch trn loss = 0.306, accuracy = 0.880
2017-10-29 15:54:41 Iter 300: batch trn loss = 0.385, accuracy = 0.840
2017-10-29 15:55:15 Iter 400: batch trn loss = 0.320, accuracy = 0.860
2017-10-29 15:55:52 Iter 500: batch trn loss = 0.191, accuracy = 0.940
2017-10-29 15:56:27 Iter 600: batch trn loss = 0.373, accuracy = 0.920
2017-10-29 15:57:00 Iter 700: batch trn loss = 0.351, accuracy = 0.880
2017-10-29 15:57:33 Iter 800: batch trn loss = 0.512, accuracy = 0.860
Epoch 0: mean loss = 0.290, accuracy = 0.896
2017-10-29 15:58:06 Iter 900: batch trn loss = 0.363, accuracy = 0.820
2017-10-29 15:58:40 Iter 1000: batch trn loss = 0.248, accuracy = 0.840
2017-10-29 15:59:13 Iter 1100: batch trn loss = 0.346, accuracy = 0.860
2017-10-29 15:59:47 Iter 1200: batch trn loss = 0.129, accuracy = 0.940
2017-10-29 16:00:21 Iter 

(0.7545625, 0.80220000000000002)

In [30]:
# 25 epochs
print('train')
cnn.train(m_X_trn, y_trn, epochs=5, batch_size=50, print_every=100, plot_losses=False)
print('validation')
cnn.validate(m_X_val, y_val, epochs=1, batch_size=y_val.shape[0])

train
2017-10-29 16:18:47 Iter 0: batch trn loss = 0.268, accuracy = 0.920
2017-10-29 16:19:21 Iter 100: batch trn loss = 0.071, accuracy = 0.960
2017-10-29 16:19:54 Iter 200: batch trn loss = 0.121, accuracy = 0.940
2017-10-29 16:20:27 Iter 300: batch trn loss = 0.147, accuracy = 0.960
2017-10-29 16:21:00 Iter 400: batch trn loss = 0.162, accuracy = 0.940
2017-10-29 16:21:34 Iter 500: batch trn loss = 0.143, accuracy = 0.960
2017-10-29 16:22:08 Iter 600: batch trn loss = 0.077, accuracy = 0.980
2017-10-29 16:22:44 Iter 700: batch trn loss = 0.201, accuracy = 0.960
2017-10-29 16:23:19 Iter 800: batch trn loss = 0.207, accuracy = 0.900
Epoch 0: mean loss = 0.203, accuracy = 0.927
2017-10-29 16:23:52 Iter 900: batch trn loss = 0.252, accuracy = 0.940
2017-10-29 16:24:26 Iter 1000: batch trn loss = 0.190, accuracy = 0.940
2017-10-29 16:24:59 Iter 1100: batch trn loss = 0.207, accuracy = 0.940
2017-10-29 16:25:33 Iter 1200: batch trn loss = 0.176, accuracy = 0.900
2017-10-29 16:26:06 Iter 

(0.79273933, 0.79679999999999995)

In [31]:
# 30 epochs
print('train')
cnn.train(m_X_trn, y_trn, epochs=5, batch_size=50, print_every=100, plot_losses=False)
print('validation')
cnn.validate(m_X_val, y_val, epochs=1, batch_size=y_val.shape[0])

train
2017-10-29 17:17:07 Iter 0: batch trn loss = 0.086, accuracy = 0.980
2017-10-29 17:17:40 Iter 100: batch trn loss = 0.197, accuracy = 0.940
2017-10-29 17:18:13 Iter 200: batch trn loss = 0.107, accuracy = 0.980
2017-10-29 17:18:45 Iter 300: batch trn loss = 0.257, accuracy = 0.860
2017-10-29 17:19:18 Iter 400: batch trn loss = 0.142, accuracy = 0.960
2017-10-29 17:19:50 Iter 500: batch trn loss = 0.284, accuracy = 0.900
2017-10-29 17:20:24 Iter 600: batch trn loss = 0.243, accuracy = 0.920
2017-10-29 17:20:57 Iter 700: batch trn loss = 0.089, accuracy = 0.980
2017-10-29 17:21:31 Iter 800: batch trn loss = 0.206, accuracy = 0.920
Epoch 0: mean loss = 0.147, accuracy = 0.947
2017-10-29 17:22:04 Iter 900: batch trn loss = 0.014, accuracy = 1.000
2017-10-29 17:22:37 Iter 1000: batch trn loss = 0.042, accuracy = 1.000
2017-10-29 17:23:10 Iter 1100: batch trn loss = 0.062, accuracy = 0.980
2017-10-29 17:23:43 Iter 1200: batch trn loss = 0.112, accuracy = 0.960
2017-10-29 17:24:16 Iter 

(0.86955673, 0.80159999999999998)

In [32]:
# 35 epochs
print('train')
cnn.train(m_X_trn, y_trn, epochs=5, batch_size=50, print_every=100, plot_losses=False)
print('validation')
cnn.validate(m_X_val, y_val, epochs=1, batch_size=y_val.shape[0])

train
2017-10-29 18:43:59 Iter 0: batch trn loss = 0.073, accuracy = 0.980
2017-10-29 18:44:31 Iter 100: batch trn loss = 0.135, accuracy = 0.960
2017-10-29 18:45:03 Iter 200: batch trn loss = 0.019, accuracy = 1.000
2017-10-29 18:45:36 Iter 300: batch trn loss = 0.039, accuracy = 0.980
2017-10-29 18:46:08 Iter 400: batch trn loss = 0.072, accuracy = 0.980
2017-10-29 18:46:40 Iter 500: batch trn loss = 0.268, accuracy = 0.900
2017-10-29 18:47:12 Iter 600: batch trn loss = 0.011, accuracy = 1.000
2017-10-29 18:47:44 Iter 700: batch trn loss = 0.275, accuracy = 0.820
2017-10-29 18:48:17 Iter 800: batch trn loss = 0.061, accuracy = 0.980
Epoch 0: mean loss = 0.105, accuracy = 0.962
2017-10-29 18:48:49 Iter 900: batch trn loss = 0.059, accuracy = 0.980
2017-10-29 18:49:21 Iter 1000: batch trn loss = 0.180, accuracy = 0.900
2017-10-29 18:49:53 Iter 1100: batch trn loss = 0.089, accuracy = 0.960
2017-10-29 18:50:25 Iter 1200: batch trn loss = 0.121, accuracy = 0.940
2017-10-29 18:50:57 Iter 

(0.88099295, 0.7994)

## [110] Base + Normalization

In [37]:
def model_fn(layer_input, labels, is_training):
    F1 = 32
    layer = tf.layers.conv2d(inputs=layer_input, filters=F1, kernel_size=[5, 5], padding='same', activation=tf.nn.relu)
    layer = tf.layers.batch_normalization(inputs=layer, training=is_training)
    layer = tf.layers.max_pooling2d(inputs=layer, pool_size=[2, 2], strides=2)
    F2 = 64
    layer = tf.layers.conv2d(inputs=layer, filters=F2, kernel_size=[5, 5], padding='same', activation=tf.nn.relu)
    layer = tf.layers.batch_normalization(inputs=layer, training=is_training)
    layer = tf.layers.max_pooling2d(inputs=layer, pool_size=[2, 2], strides=2)
    flat_size = int(H / 4 * W / 4 * F2)
    print(flat_size)
    layer = tf.reshape(layer, [-1, flat_size])
    layer = tf.layers.dense(inputs=layer, units=1024)
    layer = tf.layers.batch_normalization(inputs=layer, training=is_training)
    layer_logit = tf.layers.dense(inputs=layer, units=T)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int64), depth=10)
    loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits=layer_logit)
    return layer_logit, loss

trainer = tf.train.GradientDescentOptimizer(learning_rate=0.03)
cnn = CNN(model_fn, trainer)

4096


In [38]:
for i in range(15):
    print('train: epoch %d' % i)
    cnn.train(m_X_trn, y_trn, epochs=1, batch_size=50, print_every=100, plot_losses=False)
    print('validation')
    cnn.validate(m_X_val, y_val, epochs=1, batch_size=y_val.shape[0])

train: epoch 0
2017-10-29 22:46:42 Iter 0: batch trn loss = 3.061, accuracy = 0.120
2017-10-29 22:47:02 Iter 100: batch trn loss = 1.623, accuracy = 0.520
2017-10-29 22:47:22 Iter 200: batch trn loss = 1.551, accuracy = 0.440
2017-10-29 22:47:42 Iter 300: batch trn loss = 1.611, accuracy = 0.480
2017-10-29 22:48:02 Iter 400: batch trn loss = 0.991, accuracy = 0.680
2017-10-29 22:48:21 Iter 500: batch trn loss = 1.343, accuracy = 0.520
2017-10-29 22:48:42 Iter 600: batch trn loss = 1.673, accuracy = 0.360
2017-10-29 22:49:02 Iter 700: batch trn loss = 1.418, accuracy = 0.560
2017-10-29 22:49:22 Iter 800: batch trn loss = 1.172, accuracy = 0.560
Epoch 0: mean loss = 1.384, accuracy = 0.543
validation
Epoch 0: mean loss = 1.108, accuracy = 0.624
train: epoch 1
2017-10-29 22:49:50 Iter 0: batch trn loss = 0.684, accuracy = 0.740
2017-10-29 22:50:10 Iter 100: batch trn loss = 1.314, accuracy = 0.580
2017-10-29 22:50:30 Iter 200: batch trn loss = 1.099, accuracy = 0.660
2017-10-29 22:50:50 I

Epoch 0: mean loss = 1.168, accuracy = 0.702
train: epoch 11
2017-10-29 23:21:08 Iter 0: batch trn loss = 0.373, accuracy = 0.840
2017-10-29 23:21:29 Iter 100: batch trn loss = 0.208, accuracy = 0.920
2017-10-29 23:21:50 Iter 200: batch trn loss = 0.293, accuracy = 0.880
2017-10-29 23:22:10 Iter 300: batch trn loss = 0.270, accuracy = 0.920
2017-10-29 23:22:30 Iter 400: batch trn loss = 0.395, accuracy = 0.840
2017-10-29 23:22:51 Iter 500: batch trn loss = 0.489, accuracy = 0.800
2017-10-29 23:23:11 Iter 600: batch trn loss = 0.256, accuracy = 0.940
2017-10-29 23:23:32 Iter 700: batch trn loss = 0.250, accuracy = 0.900
2017-10-29 23:23:53 Iter 800: batch trn loss = 0.560, accuracy = 0.820
Epoch 0: mean loss = 0.351, accuracy = 0.876
validation
Epoch 0: mean loss = 1.197, accuracy = 0.702
train: epoch 12
2017-10-29 23:24:21 Iter 0: batch trn loss = 0.140, accuracy = 0.960
2017-10-29 23:24:42 Iter 100: batch trn loss = 0.371, accuracy = 0.900
2017-10-29 23:25:02 Iter 200: batch trn loss 

## [0111]
- base (0110)
- add l2 regularization 0.1

In [39]:
def model_fn(layer_input, labels, is_training):
    reg_scale= 0.1
    regularizer = tf.contrib.layers.l2_regularizer(scale=reg_scale)
    F1 = 32
    layer = tf.layers.conv2d(inputs=layer_input, filters=F1, kernel_size=[5, 5], padding='same', activation=tf.nn.relu, 
                             kernel_regularizer=regularizer)
    layer = tf.layers.batch_normalization(inputs=layer, training=is_training)
    layer = tf.layers.max_pooling2d(inputs=layer, pool_size=[2, 2], strides=2)
    F2 = 64
    layer = tf.layers.conv2d(inputs=layer, filters=F2, kernel_size=[5, 5], padding='same', activation=tf.nn.relu,
                            kernel_regularizer=regularizer)
    layer = tf.layers.batch_normalization(inputs=layer, training=is_training)
    layer = tf.layers.max_pooling2d(inputs=layer, pool_size=[2, 2], strides=2)
    flat_size = int(H / 4 * W / 4 * F2)
    print(flat_size)
    layer = tf.reshape(layer, [-1, flat_size])
    layer = tf.layers.dense(inputs=layer, units=1024,
                           kernel_regularizer=regularizer)
    layer = tf.layers.batch_normalization(inputs=layer, training=is_training)
    layer_logit = tf.layers.dense(inputs=layer, units=T,
                                 kernel_regularizer=regularizer)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int64), depth=10)
    loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits=layer_logit)
    return layer_logit, loss

trainer = tf.train.GradientDescentOptimizer(learning_rate=0.03)
cnn = CNN(model_fn, trainer)

4096


In [40]:
for i in range(15):
    print('train: epoch %d' % i)
    cnn.train(m_X_trn, y_trn, epochs=1, batch_size=50, print_every=100, plot_losses=False)
    print('validation')
    cnn.validate(m_X_val, y_val, epochs=1, batch_size=y_val.shape[0])

train: epoch 0
2017-10-29 23:36:22 Iter 0: batch trn loss = 2.905, accuracy = 0.080
2017-10-29 23:36:44 Iter 100: batch trn loss = 2.114, accuracy = 0.340
2017-10-29 23:37:04 Iter 200: batch trn loss = 1.560, accuracy = 0.480
2017-10-29 23:37:23 Iter 300: batch trn loss = 1.249, accuracy = 0.500
2017-10-29 23:37:43 Iter 400: batch trn loss = 1.528, accuracy = 0.420
2017-10-29 23:38:03 Iter 500: batch trn loss = 1.413, accuracy = 0.500
2017-10-29 23:38:23 Iter 600: batch trn loss = 1.186, accuracy = 0.640
2017-10-29 23:38:42 Iter 700: batch trn loss = 1.071, accuracy = 0.680
2017-10-29 23:39:02 Iter 800: batch trn loss = 1.012, accuracy = 0.660
Epoch 0: mean loss = 1.389, accuracy = 0.546
validation
Epoch 0: mean loss = 1.060, accuracy = 0.633
train: epoch 1
2017-10-29 23:39:30 Iter 0: batch trn loss = 0.806, accuracy = 0.680
2017-10-29 23:39:50 Iter 100: batch trn loss = 0.926, accuracy = 0.680
2017-10-29 23:40:10 Iter 200: batch trn loss = 1.099, accuracy = 0.640
2017-10-29 23:40:29 I

Epoch 0: mean loss = 1.212, accuracy = 0.685
train: epoch 11
2017-10-30 00:11:21 Iter 0: batch trn loss = 0.412, accuracy = 0.840
2017-10-30 00:11:42 Iter 100: batch trn loss = 0.109, accuracy = 0.960
2017-10-30 00:12:03 Iter 200: batch trn loss = 0.399, accuracy = 0.840
2017-10-30 00:12:25 Iter 300: batch trn loss = 0.188, accuracy = 0.960
2017-10-30 00:12:47 Iter 400: batch trn loss = 0.235, accuracy = 0.920
2017-10-30 00:13:07 Iter 500: batch trn loss = 0.335, accuracy = 0.900
2017-10-30 00:13:28 Iter 600: batch trn loss = 0.501, accuracy = 0.800
2017-10-30 00:13:49 Iter 700: batch trn loss = 0.434, accuracy = 0.880
2017-10-30 00:14:09 Iter 800: batch trn loss = 0.381, accuracy = 0.920
Epoch 0: mean loss = 0.342, accuracy = 0.880
validation
Epoch 0: mean loss = 1.197, accuracy = 0.701
train: epoch 12
2017-10-30 00:14:38 Iter 0: batch trn loss = 0.363, accuracy = 0.860
2017-10-30 00:14:59 Iter 100: batch trn loss = 0.208, accuracy = 0.900
2017-10-30 00:15:20 Iter 200: batch trn loss 

## [0111.1]
- reg 0.1 -> 1.0

In [ ]:
def model_fn(layer_input, labels, is_training):
    reg_scale= 1.0
    regularizer = tf.contrib.layers.l2_regularizer(scale=reg_scale)
    F1 = 32
    layer = tf.layers.conv2d(inputs=layer_input, filters=F1, kernel_size=[5, 5], padding='same', activation=tf.nn.relu, 
                             kernel_regularizer=regularizer)
    layer = tf.layers.batch_normalization(inputs=layer, training=is_training)
    layer = tf.layers.max_pooling2d(inputs=layer, pool_size=[2, 2], strides=2)
    F2 = 64
    layer = tf.layers.conv2d(inputs=layer, filters=F2, kernel_size=[5, 5], padding='same', activation=tf.nn.relu,
                            kernel_regularizer=regularizer)
    layer = tf.layers.batch_normalization(inputs=layer, training=is_training)
    layer = tf.layers.max_pooling2d(inputs=layer, pool_size=[2, 2], strides=2)
    flat_size = int(H / 4 * W / 4 * F2)
    print(flat_size)
    layer = tf.reshape(layer, [-1, flat_size])
    layer = tf.layers.dense(inputs=layer, units=1024,
                           kernel_regularizer=regularizer)
    layer = tf.layers.batch_normalization(inputs=layer, training=is_training)
    layer_logit = tf.layers.dense(inputs=layer, units=T,
                                 kernel_regularizer=regularizer)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int64), depth=10)
    loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits=layer_logit)
    return layer_logit, loss

trainer = tf.train.GradientDescentOptimizer(learning_rate=0.03)
cnn = CNN(model_fn, trainer)

4096


In [ ]:
for i in range(15):
    print('train: epoch %d' % i)
    cnn.train(m_X_trn, y_trn, epochs=1, batch_size=50, print_every=100, plot_losses=False)
    print('validation')
    cnn.validate(m_X_val, y_val, epochs=1, batch_size=y_val.shape[0])

train: epoch 0
2017-10-30 00:25:00 Iter 0: batch trn loss = 2.890, accuracy = 0.140
2017-10-30 00:25:20 Iter 100: batch trn loss = 1.105, accuracy = 0.640
2017-10-30 00:25:40 Iter 200: batch trn loss = 1.890, accuracy = 0.360
2017-10-30 00:26:00 Iter 300: batch trn loss = 0.996, accuracy = 0.640
2017-10-30 00:26:20 Iter 400: batch trn loss = 1.493, accuracy = 0.520
2017-10-30 00:26:40 Iter 500: batch trn loss = 1.468, accuracy = 0.520
2017-10-30 00:27:01 Iter 600: batch trn loss = 1.331, accuracy = 0.460
2017-10-30 00:27:22 Iter 700: batch trn loss = 1.409, accuracy = 0.580
2017-10-30 00:27:42 Iter 800: batch trn loss = 0.965, accuracy = 0.640
Epoch 0: mean loss = 1.403, accuracy = 0.536
validation
Epoch 0: mean loss = 1.170, accuracy = 0.597
train: epoch 1
2017-10-30 00:28:09 Iter 0: batch trn loss = 1.005, accuracy = 0.600
2017-10-30 00:28:30 Iter 100: batch trn loss = 1.107, accuracy = 0.620
2017-10-30 00:28:51 Iter 200: batch trn loss = 1.291, accuracy = 0.600
2017-10-30 00:29:11 I

## [0112]
- base (0110)
- first combination from conv-bn-pool --> conv-pool-bn

In [ ]:
def model_fn(layer_input, labels, is_training):
    reg_scale= 0.1
    regularizer = tf.contrib.layers.l2_regularizer(scale=reg_scale)
    regularizer = None
    F1 = 32
    layer = tf.layers.conv2d(inputs=layer_input, filters=F1, kernel_size=[5, 5], padding='same', activation=tf.nn.relu, 
                             kernel_regularizer=regularizer)
    layer = tf.layers.max_pooling2d(inputs=layer, pool_size=[2, 2], strides=2)
    layer = tf.layers.batch_normalization(inputs=layer, training=is_training)
    F2 = 64
    layer = tf.layers.conv2d(inputs=layer, filters=F2, kernel_size=[5, 5], padding='same', activation=tf.nn.relu,
                            kernel_regularizer=regularizer)
    layer = tf.layers.batch_normalization(inputs=layer, training=is_training)
    layer = tf.layers.max_pooling2d(inputs=layer, pool_size=[2, 2], strides=2)
    flat_size = int(H / 4 * W / 4 * F2)
    print(flat_size)
    layer = tf.reshape(layer, [-1, flat_size])
    layer = tf.layers.dense(inputs=layer, units=1024,
                           kernel_regularizer=regularizer)
    layer = tf.layers.batch_normalization(inputs=layer, training=is_training)
    layer_logit = tf.layers.dense(inputs=layer, units=T,
                                 kernel_regularizer=regularizer)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int64), depth=10)
    loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits=layer_logit)
    return layer_logit, loss

trainer = tf.train.GradientDescentOptimizer(learning_rate=0.03)
cnn = CNN(model_fn, trainer)

In [ ]:
for i in range(15):
    print('train: epoch %d' % i)
    cnn.train(m_X_trn, y_trn, epochs=1, batch_size=50, print_every=100, plot_losses=False)
    print('validation')
    cnn.validate(m_X_val, y_val, epochs=1, batch_size=y_val.shape[0])

## [0113]
- base (0112)
- F1: 32 --> 64

In [ ]:
def model_fn(layer_input, labels, is_training):
    reg_scale= 0.1
    regularizer = tf.contrib.layers.l2_regularizer(scale=reg_scale)
    regularizer = None
    F1 = 64
    layer = tf.layers.conv2d(inputs=layer_input, filters=F1, kernel_size=[5, 5], padding='same', activation=tf.nn.relu, 
                             kernel_regularizer=regularizer)
    layer = tf.layers.max_pooling2d(inputs=layer, pool_size=[2, 2], strides=2)
    layer = tf.layers.batch_normalization(inputs=layer, training=is_training)
    F2 = 64
    layer = tf.layers.conv2d(inputs=layer, filters=F2, kernel_size=[5, 5], padding='same', activation=tf.nn.relu,
                            kernel_regularizer=regularizer)
    layer = tf.layers.batch_normalization(inputs=layer, training=is_training)
    layer = tf.layers.max_pooling2d(inputs=layer, pool_size=[2, 2], strides=2)
    flat_size = int(H / 4 * W / 4 * F2)
    print(flat_size)
    layer = tf.reshape(layer, [-1, flat_size])
    layer = tf.layers.dense(inputs=layer, units=1024,
                           kernel_regularizer=regularizer)
    layer = tf.layers.batch_normalization(inputs=layer, training=is_training)
    layer_logit = tf.layers.dense(inputs=layer, units=T,
                                 kernel_regularizer=regularizer)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int64), depth=10)
    loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits=layer_logit)
    return layer_logit, loss

trainer = tf.train.GradientDescentOptimizer(learning_rate=0.03)
cnn = CNN(model_fn, trainer)

In [ ]:
for i in range(15):
    print('train: epoch %d' % i)
    cnn.train(m_X_trn, y_trn, epochs=1, batch_size=50, print_every=100, plot_losses=False)
    print('validation')
    cnn.validate(m_X_val, y_val, epochs=1, batch_size=y_val.shape[0])